<div style="text-align: center; color: red;">This is centered red text</div>


TBALE OF CONTENTS

<a name="1"></a>
## <font color='blue'> 1. Introduction </font>

This notebook explores how to build custom models in TensorFlow, focusing on implementing custom training loops. Additionally, we delve into Graph Convolutional Networks (GCNs) and their applications.

To put theory into practice, we implement a GCN model for the Karate Club dataset, a classic benchmark for graph-based learning. This will help reinforce key concepts such as:

- Building models beyond Keras' predefined layers
- Writing custom training loops for greater flexibility
- Understanding and implementing Graph Convolutional Networks

<a name="2"></a>
## <font color='blue'> 2. Setup </font>

In [1]:
#!pip install spektral

In [55]:
import numpy as np
import networkx as nx
import tensorflow as tf
from spektral.data import Dataset, Graph
from spektral.layers import GCNConv
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import scipy.sparse as sp
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Layer

In [5]:
SEED = 42

# Set seeds for reproducibility
tf.random.set_seed(SEED)
np.random.seed(SEED)

<a name="3"></a>
## <font color='blue'> 3.  Data </font>

We will use The Karate Club dataset, which is a well-known social network dataset that represents friendships in a karate club. It consists of 34 nodes (members of the club) and 78 edges (friendships).


In [123]:
# Load the Karate Club dataset (networkx)
G = nx.karate_club_graph()

<a name="3.1"></a>
### <font color='#1f618d'> 3.1.  Exploration </font>



<a name="3.2"></a>
### <font color='#1f618d'> 3.2. Pre-processing</font>

In [124]:
# number of nodes
num_nodes = len(G)
num_nodes

34

In [125]:
# Compute adjacency matrix and normalize it
#adj = nx.adjacency_matrix(G).astype(np.float32)
adj = (nx.adjacency_matrix(G) > 0).astype(np.float32)
adj += sp.eye(adj.shape[0])  # Add self-loops
degree_matrix = np.array(adj.sum(axis=1)).flatten()
D_inv_sqrt = np.diag(1.0 / np.sqrt(degree_matrix))
adj_normalized = D_inv_sqrt @ adj @ D_inv_sqrt
adj = tf.convert_to_tensor(adj_normalized, dtype=tf.float32)

In [126]:
adj.shape

TensorShape([34, 34])

### Features

In [127]:
# Compute node features
one_hot_features = np.eye(num_nodes, dtype=np.float32)
degrees = np.array([G.degree(n) for n in G.nodes()], dtype=np.float32).reshape(-1, 1)
clustering_coeffs = np.array([nx.clustering(G, n) for n in G.nodes()], dtype=np.float32).reshape(-1, 1)
betweenness = np.array(list(nx.betweenness_centrality(G).values()), dtype=np.float32).reshape(-1, 1)
pagerank = np.array(list(nx.pagerank(G).values()), dtype=np.float32).reshape(-1, 1)

In [128]:
degrees

array([[16.],
       [ 9.],
       [10.],
       [ 6.],
       [ 3.],
       [ 4.],
       [ 4.],
       [ 4.],
       [ 5.],
       [ 2.],
       [ 3.],
       [ 1.],
       [ 2.],
       [ 5.],
       [ 2.],
       [ 2.],
       [ 2.],
       [ 2.],
       [ 2.],
       [ 3.],
       [ 2.],
       [ 2.],
       [ 2.],
       [ 5.],
       [ 3.],
       [ 3.],
       [ 2.],
       [ 4.],
       [ 3.],
       [ 4.],
       [ 4.],
       [ 6.],
       [12.],
       [17.]], dtype=float32)

In [129]:
# Standardize numerical features
scaler = StandardScaler() # Standardize features by removing the mean and scaling to unit variance.
        # (z-u)/s
    # where u is the mean of the training samples or zero if with_mean=False, 
    #and s is the standard deviation of the training samples or one if with_std=False.
numeric_features = scaler.fit_transform(np.hstack([degrees, clustering_coeffs, betweenness, pagerank]))

In [130]:
# Access mean and standard deviation
print("Feature Means:", scaler.mean_)  # Mean of each column before scaling
print("Feature Standard Deviations:", scaler.scale_)  # Standard deviation of each column before scaling

Feature Means: [4.58823529 0.57063848 0.04400624 0.02941176]
Feature Standard Deviations: [3.82036068 0.34226601 0.09254295 0.02236985]


In [131]:
# Extract the first column (standardized degrees)
standardized_degrees = numeric_features[:, 0]
standardized_degrees

array([ 2.987091  ,  1.1548032 ,  1.4165585 ,  0.36953703, -0.41572914,
       -0.15397376, -0.15397376, -0.15397376,  0.10778163, -0.6774846 ,
       -0.41572914, -0.93924   , -0.6774846 ,  0.10778163, -0.6774846 ,
       -0.6774846 , -0.6774846 , -0.6774846 , -0.6774846 , -0.41572914,
       -0.6774846 , -0.6774846 , -0.6774846 ,  0.10778163, -0.41572914,
       -0.41572914, -0.6774846 , -0.15397376, -0.41572914, -0.15397376,
       -0.15397376,  0.36953703,  1.9400693 ,  3.2488465 ], dtype=float32)

In [132]:
# Concatenate all features
features = np.hstack([one_hot_features, numeric_features])
features = tf.convert_to_tensor(features, dtype=tf.float32)

In [133]:
features.shape

TensorShape([34, 38])

In [134]:
features[0].shape

TensorShape([38])

In [135]:
features[0] # one hot for nodes, and then other 4 features (degrees......)

<tf.Tensor: shape=(38,), dtype=float32, numpy=
array([ 1.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  2.987091 ,
       -1.2289811,  4.253474 ,  2.6417842], dtype=float32)>

In [136]:
# Convert labels (0 for 'Mr. Hi' club, 1 for 'Officer' club)
labels = np.array([G.nodes[i]['club'] != 'Mr. Hi' for i in range(num_nodes)], dtype=np.int32)

In [137]:
labels.shape

(34,)

In [138]:
labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

### Train/Test Split

In [139]:
# Train/test split
train_mask = np.zeros(num_nodes, dtype=bool)
train_mask[:17] = True  # First 17 nodes for training; arbitrary value
test_mask = ~train_mask
train_mask = tf.convert_to_tensor(train_mask)
test_mask = tf.convert_to_tensor(test_mask)

In [140]:
train_mask # first 17s are true and the other false

<tf.Tensor: shape=(34,), dtype=bool, numpy=
array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False])>

In [141]:
test_mask # last 34-17=17 are true, and the other are false

<tf.Tensor: shape=(34,), dtype=bool, numpy=
array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])>

<a name="4"></a>
## <font color='blue'> 4. Model </font>

We will build the following model:

........................


this model can be used to predict group memberships of each club member. The goal is to classify nodes into two groups

First, let's implement the GCN Layer:

In [142]:
# Define GCN layer
class GCNLayer(Layer):
    """
    A Graph Convolutional Network (GCN) layer.

    This layer performs graph convolution by aggregating information from neighboring nodes 
    using the adjacency matrix. It applies a linear transformation followed by an optional 
    activation function.

    Args:
        units (int): Number of output features.
        activation (str or callable, optional): Activation function to apply. Default is None.
        l2_reg (float, optional): L2 regularization factor for weight decay. Default is 1e-3.
    """

    def __init__(self, units, activation=None, l2_reg=1e-3, **kwargs):
        super(GCNLayer, self).__init__(**kwargs)
        self.units = units  # Number of output features
        self.activation = tf.keras.activations.get(activation)  # Activation function
        self.l2_reg = l2_reg  # L2 regularization factor

    def build(self, input_shape):
        """Initialize the layer's weights."""
        num_features = input_shape[0][-1]  # Get the number of input features

        # Weight matrix for feature transformation
        self.w = self.add_weight(shape=(num_features, self.units),
                                 initializer='glorot_uniform',
                                 regularizer=tf.keras.regularizers.l2(self.l2_reg),
                                 trainable=True)

        # Bias term
        self.b = self.add_weight(shape=(self.units,), initializer='zeros', trainable=True)

    def call(self, inputs):
        """
        Forward pass of the GCN layer.

        Args:
            inputs (tuple): A tuple containing:
                - features (tf.Tensor): Node feature matrix of shape (num_nodes, num_features).
                - adj (tf.Tensor): Adjacency matrix of shape (num_nodes, num_nodes).

        Returns:
            tf.Tensor: The transformed node features after applying graph convolution.
        """
        features, adj = inputs  # Unpack inputs: node features and adjacency matrix

        # Apply linear transformation to node features
        support = tf.matmul(features, self.w)

        # Aggregate neighbor information using adjacency matrix
        adj_features = tf.matmul(adj, support)

        # Add bias term
        output = adj_features + self.b

        # Apply activation function (if specified)
        return self.activation(output) if self.activation else output


Now, let's implement the entire model:

In [143]:
# Define GCN Model
class GCNModel(tf.keras.Model):
    """
    A Graph Convolutional Network (GCN) model using two GCN layers.

    This model consists of:
    - Two GCN layers for message passing
    - Batch normalization for stable training
    - Dropout for regularization

    Args:
        hidden_units (int): Number of hidden units in the first GCN layer.
        num_classes (int): Number of output classes (logits).
        dropout_rate (float, optional): Dropout rate for regularization. Default is 0.2.
    """

    def __init__(self, hidden_units, num_classes, dropout_rate=0.2):
        super(GCNModel, self).__init__()
        self.gcn1 = GCNLayer(hidden_units, activation='relu')  # First GCN layer with ReLU activation
        self.dropout1 = tf.keras.layers.Dropout(dropout_rate)  # Dropout after first layer
        self.gcn2 = GCNLayer(num_classes)  # Second GCN layer (outputs raw logits)
        self.dropout2 = tf.keras.layers.Dropout(dropout_rate)  # Dropout after second layer
        self.batch_norm = tf.keras.layers.BatchNormalization()  # Batch normalization for stability

    def call(self, inputs, training=False):
        """
        Forward pass of the GCN model.

        Args:
            inputs (tuple): A tuple containing:
                - features (tf.Tensor): Node feature matrix of shape (num_nodes, num_features).
                - adj (tf.Tensor): Adjacency matrix of shape (num_nodes, num_nodes).
            training (bool, optional): Whether the model is in training mode (for dropout and batch norm). Default is False.

        Returns:
            tf.Tensor: The output logits of shape (num_nodes, num_classes).
        """
        features, adj = inputs  # Unpack inputs: node features and adjacency matrix

        # First GCN layer with batch normalization and dropout
        x = self.gcn1((features, adj))
        x = self.batch_norm(x, training=training)  # Batch norm active only in training mode
        x = self.dropout1(x, training=training)  # Apply dropout if training

        # Second GCN layer followed by dropout
        x = self.gcn2((x, adj))
        x = self.dropout2(x, training=training)  # Apply dropout if training

        return x  # Return logits


In [144]:
# Instantiate model
model = GCNModel(hidden_units=16, num_classes=2, dropout_rate=0.2)

<a name="5"></a>
## <font color='blue'> 5. Training </font>

In [145]:
# loss function
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) # the model returns logits

In [146]:
# optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)

### Custom training loop

In [147]:
# Training loop
epochs = 500
best_accuracy = 0
for epoch in range(epochs):
    with tf.GradientTape() as tape:
        logits = model((features, adj), training=True)
        loss_value = loss_fn(labels[train_mask], logits[train_mask])
    
    grads = tape.gradient(loss_value, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # Compute test accuracy
    logits = model((features, adj), training=False)
    predictions = tf.argmax(logits, axis=1, output_type=tf.int32)
    correct_predictions = tf.equal(predictions[test_mask], labels[test_mask])
    accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32)).numpy()

    if accuracy > best_accuracy:
        best_accuracy = accuracy

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss_value.numpy()}, Test Accuracy: {accuracy}")

Epoch 0, Loss: 0.9477463960647583, Test Accuracy: 0.47058823704719543
Epoch 10, Loss: 0.305582195520401, Test Accuracy: 0.1764705926179886
Epoch 20, Loss: 0.16731642186641693, Test Accuracy: 0.1764705926179886
Epoch 30, Loss: 0.07087139040231705, Test Accuracy: 0.1764705926179886
Epoch 40, Loss: 0.1343555897474289, Test Accuracy: 0.1764705926179886
Epoch 50, Loss: 0.12809759378433228, Test Accuracy: 0.1764705926179886
Epoch 60, Loss: 0.09694959968328476, Test Accuracy: 0.1764705926179886
Epoch 70, Loss: 0.057687193155288696, Test Accuracy: 0.1764705926179886
Epoch 80, Loss: 0.05998469144105911, Test Accuracy: 0.1764705926179886
Epoch 90, Loss: 0.04889430105686188, Test Accuracy: 0.1764705926179886
Epoch 100, Loss: 0.010954556986689568, Test Accuracy: 0.1764705926179886
Epoch 110, Loss: 0.015112942084670067, Test Accuracy: 0.1764705926179886
Epoch 120, Loss: 0.027831045910716057, Test Accuracy: 0.1764705926179886
Epoch 130, Loss: 0.04748344421386719, Test Accuracy: 0.23529411852359772
E

In [148]:
print(f"Final Test Accuracy: {best_accuracy}")

Final Test Accuracy: 0.8823529481887817


<a name="annex"></a>
## <font color='blue'> Annex </font>

### adjancey matrix

To calculate

https://networkx.org/documentation/stable/reference/generated/networkx.linalg.graphmatrix.adjacency_matrix.html

In graph theory and computer science, an adjacency matrix is a square matrix used to represent a finite graph. The elements of the matrix indicate whether pairs of vertices are adjacent or not in the graph.

In the special case of a finite simple graph, the adjacency matrix is a (0,1)-matrix with zeros on its diagonal. If the graph is undirected (i.e. all of its edges are bidirectional), the adjacency matrix is symmetric. 

In [116]:
#adj = nx.adjacency_matrix(G).astype(np.float32)
adj = (nx.adjacency_matrix(G) > 0).astype(np.float32)
adj.toarray()


array([[0., 1., 1., ..., 1., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 1., 0.],
       ...,
       [1., 0., 0., ..., 0., 1., 1.],
       [0., 0., 1., ..., 1., 0., 1.],
       [0., 0., 0., ..., 1., 1., 0.]], dtype=float32)

In [117]:
# This will ensure the matrix represents only connections (1) or no connection (0). 

A self-loop is an edge that connects a node to itself. In an adjacency matrix, self-loops appear as 1s on the diagonal (i.e., adj[i, i] = 1).

the Karate Club graph (nx.karate_club_graph()) does not include self-loops by default. The adjacency matrix for this dataset initially has only 0s on the diagonal.

Why Add Self-Loops?
In Graph Convolutional Networks (GCNs), self-loops are often added because:

They allow each node to keep some of its original features in the message-passing process.
They stabilize learning by preventing nodes from relying only on their neighbors' featur

In [118]:
#sp.eye(adj.shape[0]) creates an identity matrix of the same size as adj, 
#meaning 1s on the diagonal and 0s elsewhere.
#Adding it to adj ensures that every node has a self-loop.


adj += sp.eye(adj.shape[0])  # Add self-loops
adj.toarray()

array([[1., 1., 1., ..., 1., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 1., 0.],
       ...,
       [1., 0., 0., ..., 1., 1., 1.],
       [0., 0., 1., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.]])

In [119]:
# note that now we have 1s in the diagonbal

This code normalizes the adjacency matrix using the Symmetric Normalized Laplacian:
    
$$
\tilde{A} = D^{-\frac{1}{2}} A D^{-\frac{1}{2}}
$$

- A is the adjancey matrix
- D is the degree matrix
- D**(-1/2) is the inverse square root of D.


The degree matrix D is a diagonal matrix where each diagonl entry Dii represents the degree (number of connections) of node i.

In [120]:
degree_matrix = np.array(adj.sum(axis=1)).flatten()
D_inv_sqrt = np.diag(1.0 / np.sqrt(degree_matrix))
adj_normalized = D_inv_sqrt @ adj @ D_inv_sqrt
adj = tf.convert_to_tensor(adj_normalized, dtype=tf.float32)

adj

<tf.Tensor: shape=(34, 34), dtype=float32, numpy=
array([[0.05882353, 0.0766965 , 0.07312724, ..., 0.09166985, 0.        ,
        0.        ],
       [0.0766965 , 0.1       , 0.09534626, ..., 0.        , 0.        ,
        0.        ],
       [0.07312724, 0.09534626, 0.09090909, ..., 0.        , 0.0836242 ,
        0.        ],
       ...,
       [0.09166985, 0.        , 0.        , ..., 0.14285715, 0.10482848,
        0.08908708],
       [0.        , 0.        , 0.0836242 , ..., 0.10482848, 0.07692308,
        0.06537204],
       [0.        , 0.        , 0.        , ..., 0.08908708, 0.06537204,
        0.05555556]], dtype=float32)>

### Features

In [149]:
one_hot_features = np.eye(num_nodes, dtype=np.float32)
one_hot_features # 

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

This creates a one-hot encoded feature matrix for a graph with num_nodes nodes. Each node is represented by a one-hot vector, meaning:

Each row corresponds to a node
Each column is a unique feature for that node
Only one element per row is 1, the rest are 0

In [152]:
# This computes the degree of each node in the graph G and stores it as a column vector
degrees = np.array([G.degree(n) for n in G.nodes()], dtype=np.float32).reshape(-1, 1)
degrees.shape, degrees

((34, 1),
 array([[16.],
        [ 9.],
        [10.],
        [ 6.],
        [ 3.],
        [ 4.],
        [ 4.],
        [ 4.],
        [ 5.],
        [ 2.],
        [ 3.],
        [ 1.],
        [ 2.],
        [ 5.],
        [ 2.],
        [ 2.],
        [ 2.],
        [ 2.],
        [ 2.],
        [ 3.],
        [ 2.],
        [ 2.],
        [ 2.],
        [ 5.],
        [ 3.],
        [ 3.],
        [ 2.],
        [ 4.],
        [ 3.],
        [ 4.],
        [ 4.],
        [ 6.],
        [12.],
        [17.]], dtype=float32))

In [156]:
# https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.cluster.clustering.html#networkx.algorithms.cluster.clustering
# This computes the clustering coefficient for each node in the graph G and stores it as a column vector

clustering_coeffs = np.array([nx.clustering(G, n) for n in G.nodes()], dtype=np.float32).reshape(-1, 1)
clustering_coeffs.shape, clustering_coeffs[:5]

((34, 1),
 array([[0.15      ],
        [0.33333334],
        [0.24444444],
        [0.6666667 ],
        [0.6666667 ]], dtype=float32))

The clustering coefficient of a node measures how tightly connected its neighbors are. 

https://en.wikipedia.org/wiki/Clustering_coefficient

In [158]:
# This computes the betweenness centrality for each node in the graph G and stores it as a column vector
betweenness = np.array(list(nx.betweenness_centrality(G).values()), dtype=np.float32).reshape(-1, 1)

betweenness.shape, betweenness[:5]

((34, 1),
 array([[0.43763527],
        [0.05393669],
        [0.1436568 ],
        [0.01190927],
        [0.00063131]], dtype=float32))

Betweenness centrality measures how often a node appears on the shortest paths between other nodes.

https://en.wikipedia.org/wiki/Betweenness_centrality

https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.betweenness_centrality.html

In [159]:
# This computes the PageRank score for each node in the graph G and stores it as a column vector
pagerank = np.array(list(nx.pagerank(G).values()), dtype=np.float32).reshape(-1, 1)
pagerank.shape, pagerank[:5]

((34, 1),
 array([[0.08850808],
        [0.05741484],
        [0.06276686],
        [0.03721208],
        [0.02050398]], dtype=float32))

PageRank is an algorithm that measures the importance of nodes in a graph, originally designed for ranking web pages.

https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html

In [ ]:
### USe of the model

In [58]:
from tensorflow.keras import Model, Input

# Define model
input_features = Input(shape=(3,))  # Input node features (3 features per node)
input_adj = Input(shape=(4,))  # Adjacency matrix row (for each node)

output = GCNLayer(units=2, activation='relu')([input_features, input_adj])
model = Model(inputs=[input_features, input_adj], outputs=output)

# Run the model
output = model([features, adj])
print("Model Output:\n", output.numpy())


Model Output:
 [[1.4336157 0.9010961]
 [2.4710479 1.7270117]
 [1.8811625 1.0903105]
 [1.6412334 1.0542084]]


## references

https://en.wikipedia.org/wiki/Adjacency_matrix